In [ ]:
# default_exp experiment.mining.ir.unsupervised.d2v

# Neural Unsupervised Approaches for SE Traceability [approach d2v]

> Just Paper. Full Experimentation. This module is dedicated to evaluate word2vec/doc2vec or any neural unsupervised approaches on traceability datasets. Consider to Copy the entire notebook for a new and separeted empirical evaluation. 
> Implementing mutual information analysis
> Author: @danaderp April 2020
> Author: @danielrc Nov 2020

In [ ]:
from ds4se.mining.ir import Doc2VecSeqVect as d2v
from ds4se.mining.ir import VectorizationType
from ds4se.mining.ir import LinkType
from ds4se.mining.ir import SoftwareArtifacts
from ds4se.mining.ir import Preprocessing
from ds4se.mining.ir import DistanceMetric
from ds4se.mining.ir import LoadLinks
from ds4se.mining.ir import SimilarityMetric

In [ ]:
from ds4se.mgmnt.prep.conv import *

In [ ]:
from prg import prg

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Artifacts Similarity with BasicSequenceVectorization

We test diferent similarities based on [blog](https://www.kdnuggets.com/2017/08/comparing-distance-measurements-python-scipy.html) and [blog2](https://www.kdnuggets.com/2019/01/comparison-text-distance-metrics.html)

# Experients Set-up

In [ ]:
path_data = '../dvc-ds4se/' #dataset path

In [ ]:
#Experiment 1.1.3 <<-- doc2vec
path_model_prefix = path_data+'models/bpe/sentencepiece/wiki_py_java_bpe_32k'
path_to_trained_model = path_data+'/models/pv/bpe32k/[doc2vec-Java-py-PVDBOW-500-20E-32k-1595514224.303453].model'
def sacp_params():
        return {
        "vectorizationType": VectorizationType.doc2vec,
        "linkType": LinkType.issue2src,
        "system": 'sacp-python-common',
        "path_to_trained_model": path_to_trained_model,
        "source_type": SoftwareArtifacts.PR.value,
        "target_type": SoftwareArtifacts.PY.value,
        "system_path_config": {
            "system_path": '/tf/data/cisco/sacp_data/[sacp-python-common-all-corpus-1596383717.992744].csv',
            "sep": '~',
            "names": ['ids','bpe32k'],
            "prep": Preprocessing.bpe
        },
        "path_mappings": "/tf/data/cisco/sacp_data/sacp-pr-mappings.csv",
        "saving_path": path_data + 'metrics/traceability/experiments1.1.x/',
        "names": ['Source','Target','Linked?'],
        "model_prefix": path_model_prefix
        }

In [ ]:

parameters = sacp_params()
parameters

{'vectorizationType': <VectorizationType.doc2vec: 2>,
 'linkType': <LinkType.issue2src: 3>,
 'system': 'sacp-python-common',
 'path_to_trained_model': '../dvc-ds4se//models/pv/bpe32k/[doc2vec-Java-py-PVDBOW-500-20E-32k-1595514224.303453].model',
 'source_type': 'pr',
 'target_type': 'py',
 'system_path_config': {'system_path': '/tf/data/cisco/sacp_data/[sacp-python-common-all-corpus-1596383717.992744].csv',
  'sep': '~',
  'names': ['ids', 'bpe32k'],
  'prep': <Preprocessing.bpe: 2>},
 'path_mappings': '/tf/data/cisco/sacp_data/sacp-pr-mappings.csv',
 'saving_path': '../dvc-ds4se/metrics/traceability/experiments1.1.x/',
 'names': ['Source', 'Target', 'Linked?'],
 'model_prefix': '../dvc-ds4se/models/bpe/sentencepiece/wiki_py_java_bpe_32k'}

# Artifacts Similarity with Doc2Vec

In [ ]:
#[step 1]Creating the Vectorization Class
doc2vec = d2v( params = parameters, logging = logging )

KeyError: "['bpe32k'] not in index"

In [ ]:
doc2vec.df_source.head(2)

In [ ]:
str(type(doc2vec.new_model))

NameError: name 'doc2vec' is not defined

In [ ]:
#[step1]Apply Doc2Vec Inference
doc2vec.InferDoc2Vec( steps = 200 )

In [ ]:
doc2vec.df_inferred_src.head(2)

In [ ]:
#[step 2]NonGroundTruth Computation
metric_l = [DistanceMetric.EUC,DistanceMetric.COS,DistanceMetric.MAN, SimilarityMetric.Pearson]
doc2vec.ComputeDistanceArtifacts( sampling=False, samples = 50, metric_list = metric_l )
doc2vec.df_nonground_link.head()

In [ ]:
#[step 3]Saving Non-GroundTruth Links
doc2vec.SaveLinks()

In [ ]:
#[step 4]GroundTruthMatching Testing
path_to_ground_truth = parameters['path_mappings']

doc2vec.MatchWithGroundTruth(path_to_ground_truth, semeru_format= True)
doc2vec.df_ground_link

In [ ]:
#[step 4.1]GroundTruthMatching Testing For CISCO Mappings <--- Only SACP
doc2vec.MatchWithGroundTruth(from_mappings=True)
doc2vec.df_ground_link

In [ ]:
#[step 5]Saving GroundTruth Links
doc2vec.SaveLinks(grtruth = True)

In [ ]:
#Loading Non-GroundTruth Links (change the timestamp with the assigned in the previous step)
df_glinks = LoadLinks(timestamp=1609004403.942551, params=parameters,grtruth = True, logging = logging)
df_glinks.head()